# Usage Instructions for This Notebook
- **Cells 2-20:** For local use only (embedding generation, upsert to Pinecone, etc.). These require a lot of memory and are not suitable for Render or low-memory environments.
- **For deployment (Render):** Only use Pinecone for retrieval and Groq for answering. Do not run embedding or torch-related cells in production.
- **See the last cell for a lightweight retrieval/QA example suitable for Render.

In [32]:
%cd ..

c:\Users\nipun\Desktop\LLmplusrag


c:\Users\nipun\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [33]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os 

In [34]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [35]:
extracted_data=load_pdf_file(data='Data/')

FileNotFoundError: Directory not found: 'Data/'

In [ ]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1063


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [ ]:
embeddings = download_hugging_face_embeddings()

C:\Users\nipun\AppData\Local\Temp\ipykernel_24612\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\nipun\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
if not GROQ_API_KEY:
    raise ValueError("Groq API Key is missing. Set it in the .env file.")

In [ ]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

# Load environment variables from .env
load_dotenv()
# Get Pinecone API key from environment
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if not PINECONE_API_KEY:
    raise ValueError("Pinecone API Key is missing. Set it in the .env file.")

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define the index name
index_name = "medbot"

# Create the index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Adjust based on your embeddings
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

print(f"Index '{index_name}' is ready!")


Index 'medbot' is ready!


In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings.base import Embeddings

# Ensure PyTorch uses GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the embedding model and move it to GPU
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")

# Custom embedding class for LangChain compatibility
class CustomEmbeddings(Embeddings):
    def embed_documents(self, texts):
        with torch.no_grad():
            return embedding_model.encode(texts, convert_to_tensor=True, device=device).cpu().numpy().tolist()

    def embed_query(self, text):
        return self.embed_documents([text])[0]

# Instantiate the custom embedding class
embedding_instance = CustomEmbeddings()

# Batch processing for Pinecone upsert
batch_size = 1000  # Adjust as needed
for i in range(0, len(text_chunks), batch_size):
    docsearch = PineconeVectorStore.from_documents(
        documents=text_chunks[i:i+batch_size],
        index_name=index_name,
        embedding=embedding_instance  # Use the instance, not a function
    )


In [ ]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
retrieved_docs = retriever.invoke("What is TIme complexity?")

In [ ]:

retrieved_docs

[Document(id='3b5f9cdc-1850-40e5-82db-afa1d0960e44', metadata={'author': 'EDGE Note', 'creationdate': '2009-08-20T02:28:03-05:00', 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2009-08-20T02:28:03-05:00', 'page': 979.0, 'page_label': '980', 'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'source': 'Data\\Medical_book.pdf', 'title': 'Principles of internal medicine', 'total_pages': 5113.0}, page_content='neoplasm is malignant. Cancer is a synonym for malignant neoplasm. Cancers of \nepithelial tissues are called carcinomas; cancers of nonepithelial (mesenchymal) tissues \nare called sarcomas.\nCancer is a genetic disease, but the level of its expression is the single cell. Although \nsome forms of cancer are heritable, most mutations occur in somatic cells and are \ncaused by intrinsic errors in DNA replication or are induced by carcinogen exposure. A'),
 Document(id='7d52f22a-2213-423a-a4fb-0c5e9333b9aa', metadata={'author': 'EDGE Note', 'creationdate': '2009-08-20T02:28:03-05:

In [ ]:
import os
from langchain_groq import ChatGroq

# Set API key directly
api_key = "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"

llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.4, max_tokens=500, api_key=api_key)


In [38]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a DSA assistant. For every question or code input, only provide the time complexity of the code in your answer. "
    "Do not provide space complexity, explanations, or any other information. "
    "Output only the time complexity in standard notation (e.g., O(n), O(1), O(n^2)). "
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])


In [39]:
# Create the question-answering chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Assume `retriever` is already defined elsewhere
rag_chain = create_retrieval_chain(retriever, question_answer_chain)  

In [40]:
# RAG + LLM QA Example
question = "Print(hello world)   ?"

response = rag_chain.invoke({"input": question})

print("Answer:", response["answer"])

Answer: O(1)


In [ ]:
# # Lightweight Retrieval/QA Example for Render (No Embedding Model, No torch)
# import os
# from dotenv import load_dotenv
# from langchain_pinecone import PineconeVectorStore
# from langchain_groq import ChatGroq
# from langchain.chains import RetrievalQA
# from langchain_core.prompts import ChatPromptTemplate
# from pinecone import Pinecone
# load_dotenv()
# PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# GROQ_API_KEY = os.getenv("GROQ_API_KEY")
# index_name = "medbot"
# pc = Pinecone(api_key=PINECONE_API_KEY)
# vector_store = PineconeVectorStore(
#     index_name=index_name,
#     embedding=None,  # No embedding model needed at runtime
#     pinecone_api_key=PINECONE_API_KEY
# )
# groq_llm = ChatGroq(model_name="mixtral-8x7b-32768", api_key=GROQ_API_KEY)
# # System prompt specialized for rainwater harvesting, but allows any question
# system_prompt = (
#     "You are an assistant specialized in rainwater harvesting and related topics. "+
#     "Use the provided context to answer user questions as thoroughly as possible. "+
#     "If the answer is not in the context, say you don't know.\n\n{context}"
# )
# prompt = ChatPromptTemplate.from_messages([
#     ("system", system_prompt),
#     ("human", "{input}"),
# ])
# retriever = vector_store.as_retriever()
# qa_chain = RetrievalQA.from_chain_type(llm=groq_llm, retriever=retriever, prompt=prompt)
# response = qa_chain.invoke({"query": "How do I implement rainwater harvesting on my rooftop in Rajasthan?"})
# print(response)

ValueError: Embedding must be provided

## Lightweight Rainwater Harvesting RAG QA (for Render/Production)
This cell demonstrates how to use your Pinecone index and Groq LLM to answer questions about rainwater harvesting (or any topic covered in your indexed documents).
- The bot is specialized for rainwater harvesting, but you can ask any question.
- No embedding model or torch is used at runtime—suitable for Render and low-memory environments.